In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell

In [59]:
# Libraries Settings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=Warning)
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
np.set_printoptions(suppress=True)

def set_seed(seed=42):
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
pd.set_option("display.width", 100)
pd.set_option("display.max_columns", 60)
pd.set_option("display.max_rows", 25)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [60]:
df_deals = pd.read_excel ('data/Deals (investment).xlsx')
df_deals.head()

,1_post_date,2_post_title,3_amount,4_Stake,5_funding_round,6_investors,7_source,Country/Town,Country(HQ),Category,main_sector,Check vs Companies,"Deals Information, Level of Completeness",duplicated_conc,Duplication,Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),"funding round, DA","Industry, DA",Year,Month,Quarter,Half,Number of Investors,Investor 1,Investor 2,Investor 3,Investor 4,Investor 5,Investor 6,Investor 7,Investor 8,Investor 9,Investor 10,Investor 11,Investor 12,Investor 13,Investor 14,Investor 15,Cummulative Deals (Disclosed),Cummulative Deals (Undisclosed),Cummulative Amount,Investor 1.1,Investor 2.1,Investor 3.1,Investor 4.1,Investor 5.1,Investor 6.1,Investor 7.1,Investor 8.1,Investor 9.1,Investor 10.1,Investor 11.1,Investor 12.1,Investor 13.1,Investor 14.1,Investor 15.1
0,2008-09-01,biNu,600000,NaN,Seed,Artesian VC,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000,9.000,3.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-11-01,biNu,400000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,Australia,Australia,Mobile Internet,Information Technology,biNu,0.900,NaN,NaN,1.000,2008.000,14220000.000,1.000,2. Seed,Other Technologies & Information Technology,2008.000,11.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-12-03,AllLife,6000000,NaN,Private Equity,LeapFrog Investments,http://www.prnewswire.co.uk/news-releases/leap...,South Africa,South Africa,"Insurance, InsurTech",Financial Services,AllLife,0.900,NaN,NaN,1.000,2004.000,12700000.000,0.000,6. Private Equity,Financial Services,2009.000,12.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-12-11,Bridge International Academies,1800000,NaN,Grant,Omidyar Network,http://foundationcenter.org/pnd/news/story.jht...,NaN,NaN,NaN,NaN,NaN,0.643,NaN,NaN,1.000,NaN,27800000.000,NaN,1. Grant,NaN,2009.000,12.000,4.000,2.000,1.000,Omidyar Network,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,Omidyar Network,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-01,biNu,320000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000,4.000,2.000,1.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(5))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df_deals)

##################### Shape #####################
(2059, 59)
##################### Types #####################
1_post_date        datetime64[ns]
2_post_title               object
3_amount                   object
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 11.1              object
Investor 12.1              object
Investor 13.1              object
Investor 14.1              object
Investor 15.1             float64
Length: 59, dtype: object
##################### Head #####################
  1_post_date                    2_post_title 3_amount  4_Stake 5_funding_round  \
0  2008-09-01                            biNu   600000      NaN            Seed   
1  2008-11-01                            biNu   400000      NaN           Angel   
2  2009-12-03                         AllLife  6000000      NaN  Private Equity   
3  2009-12-11  Bridge International Academies  1800000      NaN           Grant   
4  2010-04-01            

In [62]:
cols = df_deals.columns

In [63]:
cols

Index(['1_post_date', '2_post_title', '3_amount', '4_Stake', '5_funding_round', '6_investors',
       '7_source', 'Country/Town', 'Country(HQ)', 'Category', 'main_sector', 'Check vs Companies',
       'Deals Information, Level of Completeness', 'duplicated_conc', 'Duplication', 'Disclosed ',
       'Founded', 'Total  Disclosed Funding',
       'DA Classification_African Company (Yes = 0; No = 1)', 'funding round, DA', 'Industry, DA',
       'Year', 'Month', 'Quarter', 'Half', 'Number of Investors', 'Investor 1', 'Investor 2',
       'Investor 3', 'Investor 4', 'Investor 5', 'Investor 6', 'Investor 7', 'Investor 8',
       'Investor 9', 'Investor 10', 'Investor 11', 'Investor 12', 'Investor 13', 'Investor 14',
       'Investor 15', 'Cummulative Deals (Disclosed)', 'Cummulative Deals (Undisclosed)',
       'Cummulative Amount', 'Investor 1.1', 'Investor 2.1', 'Investor 3.1', 'Investor 4.1',
       'Investor 5.1', 'Investor 6.1', 'Investor 7.1', 'Investor 8.1', 'Investor 9.1',
       'Inv

In [64]:
for cat_col in cols:
    print(f"{cat_col}: {df_deals[cat_col].nunique()} uniqueness variable")

1_post_date: 695 uniqueness variable
2_post_title: 800 uniqueness variable
3_amount: 281 uniqueness variable
4_Stake: 7 uniqueness variable
5_funding_round: 27 uniqueness variable
6_investors: 717 uniqueness variable
7_source: 1070 uniqueness variable
Country/Town: 34 uniqueness variable
Country(HQ): 34 uniqueness variable
Category: 660 uniqueness variable
main_sector: 48 uniqueness variable
Check vs Companies: 727 uniqueness variable
Deals Information, Level of Completeness: 6 uniqueness variable
duplicated_conc: 1120 uniqueness variable
Duplication: 1 uniqueness variable
Disclosed : 2 uniqueness variable
Founded: 23 uniqueness variable
Total  Disclosed Funding: 280 uniqueness variable
DA Classification_African Company (Yes = 0; No = 1): 2 uniqueness variable
funding round, DA: 8 uniqueness variable
Industry, DA: 13 uniqueness variable
Year: 14 uniqueness variable
Month: 12 uniqueness variable
Quarter: 4 uniqueness variable
Half: 2 uniqueness variable
Number of Investors: 12 uniquenes

In [65]:
df_deals = df_deals.drop(['Investor 13.1','Investor 14.1','Investor 15.1','Investor 15','Duplication'], axis=1)